In [8]:
import requests
import json
openrouter_key='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'
response = requests.post(
  url="https://openrouter.ai/api/v1/chat/completions",
  headers={
    "Authorization": f"Bearer {openrouter_key}",
    "Content-Type": "application/json",
  },
  data=json.dumps({
    "model": "deepseek/deepseek-chat-v3.1:free",
    "messages": [
      {
        "role": "user",
        "content": "解释一下多因子策略在投资组合中的作用?用中文回答"
      }
    ],
    
  })
)

In [7]:
response.json()["choices"][0]["message"]["content"]

KeyError: 'choices'

In [5]:
import requests
import json
openrouter_key='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'
response = requests.get(
  url="https://openrouter.ai/api/v1/key",
  headers={
    "Authorization": f"Bearer {openrouter_key}"
  }
)
print(json.dumps(response.json(), indent=2))

{
  "error": {
    "message": "User not found.",
    "code": 401
  }
}


In [4]:
import pandas as pd 
df=pd.read_parquet(r'D:\projects\q\data\minute_data\000002\2025-09.parquet')

In [7]:
import sys
import os

# 添加项目路径
sys.path.append(r'D:\projects\q\myQ')
from quantlib.market_data import get_stock_data

In [11]:
from quantlib.market_data import MarketDataManager
manager = MarketDataManager()
data = manager.get_stock_data('601899', market='CN', period='1mo', interval='1d')

× 601899: 获取A股数据失败 - '1d'


ak.stock_news_em

In [16]:
import requests
import json

# 测试新浪财经新闻API
url = "https://feed.mix.sina.com.cn/api/roll/get"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'
}

params = {
    'pageid': 153,
    'lid': 2516,  # 财经新闻
    'k': '',
    'num': 10,
    'page': 1
}

print("请求URL:", url)
print("参数:", params)

response = requests.get(url, headers=headers, params=params)

print("\n状态码:", response.status_code)
print("响应长度:", len(response.text))

if response.status_code == 200:
    try:
        data = response.json()
        print("\nJSON结构:")
        print("顶级键:", list(data.keys()))
        
        if 'result' in data:
            print("result键:", list(data['result'].keys()))
            
            if 'data' in data['result']:
                news_list = data['result']['data']
                print(f"获取到 {len(news_list)} 条新闻")
                
                print("\n前3条新闻:")
                for i, news in enumerate(news_list[:3]):
                    print(f"\n新闻{i+1}:")
                    print("  字段:", list(news.keys()))
                    print("  标题:", news.get('title', ''))
                    print("  URL:", news.get('url', ''))
                    print("  时间:", news.get('ctime', ''))
                    print("  来源:", news.get('media', ''))
                    
    except json.JSONDecodeError:
        print("不是JSON格式")
        print("前500字符:", response.text[:500])
else:
    print("请求失败")
    print("响应内容:", response.text[:500])

请求URL: https://feed.mix.sina.com.cn/api/roll/get
参数: {'pageid': 153, 'lid': 2516, 'k': '', 'num': 10, 'page': 1}

状态码: 200
响应长度: 20278

JSON结构:
顶级键: ['result']
result键: ['status', 'timestamp', 'top', 'pdps', 'cre', 'total', 'end', 'start', 'lid', 'rtime', 'data']
获取到 10 条新闻

前3条新闻:

新闻1:
  字段: ['icons', 'is_cre_manual', 'hqChart', 'intime', 'channelid', 'ctime', 'mtime', 'authoruid', 'level', 'vid', 'ipad_vid', 'video_time_length', 'categoryid', 'mediaid', 'templateid', 'productid', 'ext_0', 'ext_1', 'ext_2', 'ext_3', 'ext_4', 'docid', 'url', 'urls', 'wapurl', 'wapurls', 'wapsummary', 'title', 'stitle', 'summary', 'intro', 'author', 'commentid', 'video_id', 'keywords', 'media_name', 'columnid', 'subjectid', 'img', 'images', 'lids', 'oid']
  标题: 国金证券：首予中国生物制药“买入”评级 目标价11.25港元
  URL: https://finance.sina.com.cn/tob/2025-09-23/doc-infrnksr7542561.shtml
  时间: 1758609799
  来源: 

新闻2:
  字段: ['icons', 'is_cre_manual', 'hqChart', 'intime', 'channelid', 'ctime', 'mtime', 'authoruid', 'level', '

In [20]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
import pandas as pd

def get_sina_search_news(stock_name, max_pages=3):
    """从新浪搜索获取个股新闻"""
    print(f"搜索: {stock_name}")
    
    search_url = "https://search.sina.com.cn/news"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    all_news = []
    
    for page in range(1, max_pages + 1):
        print(f"\n获取第{page}页...")
        
        params = {
            'q': stock_name,
            'c': 'news', 
            'from': 'index',
            'page': page
        }
        
        try:
            response = requests.get(search_url, headers=headers, params=params)
            
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'html.parser')
                
                # 找到所有财经新闻链接
                all_links = soup.find_all('a', href=True)
                page_news = []
                
                for link in all_links:
                    href = link.get('href', '')
                    if 'finance.sina.com.cn' in href:
                        title = link.get_text().strip()
                        if title and len(title) > 10:
                            # 避免重复
                            if not any(news['title'] == title for news in all_news):
                                news_item = {
                                    'title': title,
                                    'url': href,
                                    'date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                    'source': '新浪财经'
                                }
                                page_news.append(news_item)
                                all_news.append(news_item)
                
                print(f"第{page}页找到 {len(page_news)} 条新闻")
                for news in page_news:
                    print(f"  ✓ {news['title'][:50]}...")
                    
        except Exception as e:
            print(f"第{page}页获取失败: {e}")
    
    print(f"\n总共获取 {len(all_news)} 条新闻")
    return all_news

def get_news_content(url):
    """获取新闻内容"""
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36'}
    
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.encoding = 'utf-8'
        
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser', from_encoding='utf-8')
            
            # 移除无用标签
            for tag in soup(['script', 'style', 'iframe']):
                tag.decompose()
            
            # 尝试常见的内容选择器
            selectors = ['#artibody', '.article-body', '.article-content', '#article_content']
            
            for selector in selectors:
                content_div = soup.select_one(selector)
                if content_div:
                    text = content_div.get_text(separator=' ', strip=True)
                    if len(text) > 100:
                        return text[:2000]  # 限制长度
            
            # 备用方案：获取段落
            paragraphs = soup.find_all('p')
            if paragraphs:
                content = ' '.join([p.get_text(strip=True) for p in paragraphs if len(p.get_text(strip=True)) > 20])
                if len(content) > 100:
                    return content[:2000]
                    
    except Exception as e:
        print(f"获取内容失败: {e}")
    
    return ""

# 测试函数
def test_sina_news():
    """测试获取新闻"""
    stock_name = "紫金矿业"
    
    # 获取新闻列表
    news_list = get_sina_search_news(stock_name, max_pages=2)
    
    if news_list:
        print(f"\n=== 开始获取新闻内容 ===")
        
        # 获取每条新闻的内容
        for i, news in enumerate(news_list[:3], 1):  # 只处理前3条测试
            print(f"\n{i}. 获取内容: {news['title'][:50]}...")
            content = get_news_content(news['url'])
            
            if content:
                news['content'] = f"{news['title']}\n\n{content}"
                print(f"   ✓ 成功 ({len(content)} 字符)")
            else:
                print(f"   ✗ 失败")
        
        # 转为DataFrame
        df_data = []
        for news in news_list[:3]:
            if 'content' in news:
                df_data.append({
                    'date': news['date'],
                    'content': news['content']
                })
        
        if df_data:
            df = pd.DataFrame(df_data)
            print(f"\n=== 结果 ===")
            print(f"获得 {len(df)} 条有效新闻")
            
            # 保存测试
            df.to_csv('sina_test_news.csv', index=False, encoding='utf-8-sig')
            print("已保存到 sina_test_news.csv")
            
            return df
    
    return pd.DataFrame()

if __name__ == "__main__":
    test_sina_news()

搜索: 紫金矿业

获取第1页...
第1页找到 4 条新闻
  ✓ 大行评级｜杰富瑞：维持紫金矿业“买入”评级 分拆紫金黄金国际有助释放价值...
  ✓ 中国罕王：向紫金矿业配售融资2.35亿，拓澳洲金矿 | 港股再融资...
  ✓ 中国罕王早盘涨超19% 折让14.7%向紫金矿业发行新股...
  ✓ 紫金矿业涨2.01%，成交额11.13亿元，主力资金净流入1.14亿元...

获取第2页...
第2页找到 0 条新闻

总共获取 4 条新闻

=== 开始获取新闻内容 ===

1. 获取内容: 大行评级｜杰富瑞：维持紫金矿业“买入”评级 分拆紫金黄金国际有助释放价值...
   ✓ 成功 (348 字符)

2. 获取内容: 中国罕王：向紫金矿业配售融资2.35亿，拓澳洲金矿 | 港股再融资...
   ✓ 成功 (441 字符)

3. 获取内容: 中国罕王早盘涨超19% 折让14.7%向紫金矿业发行新股...
   ✓ 成功 (435 字符)

=== 结果 ===
获得 3 条有效新闻
已保存到 sina_test_news.csv


In [3]:
import tushare as ts 
pro = ts.pro_api()


#提取新闻内容
df = pro.news(src='sina', start_date='2025-09-21 10:00:00', end_date='2025-09-22 10:59:00')

In [4]:
df

,datetime,content,title
0,2025-09-22 10:58:05,【省级“苏超式”足球联赛持续扩容，多地创下现场观赛人数纪录】“雄起”“雄起”“雄起”——四川...,None
1,2025-09-22 10:57:39,【受台风影响，国家海洋预报台发布海浪橙色警报和风暴潮黄色警报】风暴潮方面，受今年第18号台风...,None
2,2025-09-22 10:57:04,"台湾证交所加权股价指数上涨1.1%至25,868.45点。",None
3,2025-09-22 10:56:26,澳洲联储主席布洛克：就业接近充分水平，劳动力市场略显紧张。,None
4,2025-09-22 10:56:05,【化纤板块持续拉升 吉林化纤涨停】化纤板块持续拉升，吉林化纤涨停，新乡化纤、万凯新材等跟涨。,None
...,...,...,...
897,2025-09-21 10:20:53,市场消息：卡塔尔埃米尔前往纽约参加联合国大会。,None
898,2025-09-21 10:17:05,【美新罕布什尔州俱乐部枪击事件枪手人数确定为1人 现已被捕】当地时间9月20日，当地警方通报...,None
899,2025-09-21 10:15:44,【特朗普威胁阿富汗归还空军基地】当地9月20日，美国总统特朗普在社交媒体平台“真实社交”上发...,None
900,2025-09-21 10:10:24,"【特朗普再向欧洲国家施压 要求停止购买俄罗斯石油】美国总统特朗普再次呼吁欧洲国家""停止购买""...",None


In [5]:
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta
import time

def get_stock_news(keyword, start_date, end_date, pro_api=None):
    """
    从tushare获取包含关键词的新闻
    
    Args:
        keyword: 搜索关键词
        start_date: 开始日期 '2025-01-01'
        end_date: 结束日期 '2025-09-23'  
        pro_api: tushare pro接口，如果为None则自动创建
    
    Returns:
        DataFrame: 包含date和content的新闻数据
    """
    if pro_api is None:
        pro_api = ts.pro_api()
    
    print(f"搜索关键词: {keyword}")
    print(f"时间范围: {start_date} 到 {end_date}")
    
    # 转换日期格式
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    
    all_news = []
    current_dt = start_dt
    
    while current_dt < end_dt:
        # 每次查询1小时
        next_dt = current_dt + timedelta(hours=1)
        if next_dt > end_dt:
            next_dt = end_dt
        
        start_str = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        end_str = next_dt.strftime('%Y-%m-%d %H:%M:%S')
        
        print(f"查询时段: {start_str} - {end_str}")
        
        try:
            # 调用tushare接口
            df = pro_api.news(
                src='sina', 
                start_date=start_str, 
                end_date=end_str
            )
            
            if not df.empty:
                # 在content列搜索关键词
                mask = df['content'].str.contains(keyword, na=False)
                filtered_df = df[mask]
                
                if not filtered_df.empty:
                    print(f"  找到 {len(filtered_df)} 条相关新闻")
                    all_news.append(filtered_df)
                else:
                    print(f"  无相关新闻")
            else:
                print(f"  无新闻数据")
                
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 移动到下一个时间段
        current_dt = next_dt
    
    # 合并所有结果
    if all_news:
        result_df = pd.concat(all_news, ignore_index=True)
        
        # 去重（基于title和datetime）
        result_df = result_df.drop_duplicates(subset=['title', 'datetime'])
        
        # 只保留需要的列
        final_df = result_df[['datetime', 'content']].copy()
        final_df.columns = ['date', 'content']
        
        # 按时间排序
        final_df = final_df.sort_values('date').reset_index(drop=True)
        
        print(f"\n总共获取 {len(final_df)} 条去重后的相关新闻")
        return final_df
    else:
        print("\n未找到相关新闻")
        return pd.DataFrame(columns=['date', 'content'])

# 使用示例
if __name__ == "__main__":
    # 初始化tushare
    pro = ts.pro_api()
    
    # 获取紫金矿业相关新闻
    df = get_stock_news(
        keyword="紫金矿业",
        start_date="2024-09-20", 
        end_date="2025-09-23",
        pro_api=pro
    )
    
    if not df.empty:
        print("\n=== 结果预览 ===")
        print(f"数据形状: {df.shape}")
        print(f"时间范围: {df['date'].min()} 到 {df['date'].max()}")
        
        print("\n前3条新闻:")
        for i, row in df.head(3).iterrows():
            print(f"\n{i+1}. 时间: {row['date']}")
            print(f"   内容: {row['content'][:150]}...")
        
        # 保存结果
        df.to_csv('tushare_news.csv', index=False, encoding='utf-8-sig')
        print("\n数据已保存到 tushare_news.csv")
    else:
        print("未获取到数据")

搜索关键词: 紫金矿业
时间范围: 2024-09-20 到 2025-09-23
查询时段: 2024-09-20 00:00:00 - 2024-09-20 01:00:00
  无相关新闻
查询时段: 2024-09-20 01:00:00 - 2024-09-20 02:00:00
  无相关新闻
查询时段: 2024-09-20 02:00:00 - 2024-09-20 03:00:00
  无相关新闻
查询时段: 2024-09-20 03:00:00 - 2024-09-20 04:00:00
  无相关新闻
查询时段: 2024-09-20 04:00:00 - 2024-09-20 05:00:00
  无相关新闻
查询时段: 2024-09-20 05:00:00 - 2024-09-20 06:00:00
  无相关新闻
查询时段: 2024-09-20 06:00:00 - 2024-09-20 07:00:00
  无相关新闻
查询时段: 2024-09-20 07:00:00 - 2024-09-20 08:00:00
  无相关新闻
查询时段: 2024-09-20 08:00:00 - 2024-09-20 09:00:00
  无相关新闻
查询时段: 2024-09-20 09:00:00 - 2024-09-20 10:00:00
  找到 3 条相关新闻
查询时段: 2024-09-20 10:00:00 - 2024-09-20 11:00:00
  找到 1 条相关新闻
查询时段: 2024-09-20 11:00:00 - 2024-09-20 12:00:00
  找到 2 条相关新闻
查询时段: 2024-09-20 12:00:00 - 2024-09-20 13:00:00
  无相关新闻
查询时段: 2024-09-20 13:00:00 - 2024-09-20 14:00:00
  无相关新闻
查询时段: 2024-09-20 14:00:00 - 2024-09-20 15:00:00


KeyboardInterrupt: 

In [7]:
import tushare as ts
import pandas as pd
from datetime import datetime, timedelta
import time

def get_stock_news(keyword, start_date, end_date, pro_api=None):
    """
    从tushare获取包含关键词的新闻
    
    Args:
        keyword: 搜索关键词
        start_date: 开始日期 '2025-01-01'
        end_date: 结束日期 '2025-09-23'  
        pro_api: tushare pro接口，如果为None则自动创建
    
    Returns:
        DataFrame: 包含date和content的新闻数据
    """
    if pro_api is None:
        pro_api = ts.pro_api()
    
    print(f"搜索关键词: {keyword}")
    print(f"时间范围: {start_date} 到 {end_date}")
    
    # 转换日期格式
    start_dt = datetime.strptime(start_date, '%Y-%m-%d')
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    
    all_news = []
    current_dt = start_dt
    
    while current_dt < end_dt:
        # 每次查询1天
        next_dt = current_dt + timedelta(days=1)
        if next_dt > end_dt:
            next_dt = end_dt
        
        start_str = current_dt.strftime('%Y-%m-%d 00:00:00')
        end_str = next_dt.strftime('%Y-%m-%d 00:00:00')
        
        print(f"查询时段: {start_str} - {end_str}")
        
        try:
            # 调用tushare接口
            df = pro_api.news(
                src='sina', 
                start_date=start_str, 
                end_date=end_str
            )
            
            if not df.empty:
                # 在content列搜索关键词
                mask = df['content'].str.contains(keyword, na=False)
                filtered_df = df[mask]
                
                if not filtered_df.empty:
                    print(f"  找到 {len(filtered_df)} 条相关新闻")
                    all_news.append(filtered_df)
                else:
                    print(f"  无相关新闻")
            else:
                print(f"  无新闻数据")
                
        except Exception as e:
            print(f"  查询失败: {e}")
        
        # 移动到下一个时间段
        current_dt = next_dt
    
    # 合并所有结果
    if all_news:
        result_df = pd.concat(all_news, ignore_index=True)
        
        # 去重（基于title和datetime）
        result_df = result_df.drop_duplicates(subset=['title', 'datetime'])
        
        # 只保留需要的列
        final_df = result_df[['datetime', 'content']].copy()
        final_df.columns = ['date', 'content']
        
        # 按时间排序
        final_df = final_df.sort_values('date').reset_index(drop=True)
        
        print(f"\n总共获取 {len(final_df)} 条去重后的相关新闻")
        return final_df
    else:
        print("\n未找到相关新闻")
        return pd.DataFrame(columns=['date', 'content'])

# 使用示例
if __name__ == "__main__":
    # 初始化tushare
    pro = ts.pro_api()
    
    # 获取紫金矿业相关新闻
    df = get_stock_news(
        keyword="紫金矿业",
        start_date="2024-09-20", 
        end_date="2025-09-23",
        pro_api=pro
    )
    
    if not df.empty:
        print("\n=== 结果预览 ===")
        print(f"数据形状: {df.shape}")
        print(f"时间范围: {df['date'].min()} 到 {df['date'].max()}")
        
        print("\n前3条新闻:")
        for i, row in df.head(3).iterrows():
            print(f"\n{i+1}. 时间: {row['date']}")
            print(f"   内容: {row['content'][:150]}...")
        
        # 保存结果
        df.to_csv('tushare_news.csv', index=False, encoding='utf-8-sig')
        print("\n数据已保存到 tushare_news.csv")
    else:
        print("未获取到数据")

搜索关键词: 紫金矿业
时间范围: 2024-09-20 到 2025-09-23
查询时段: 2024-09-20 00:00:00 - 2024-09-21 00:00:00
  找到 3 条相关新闻
查询时段: 2024-09-21 00:00:00 - 2024-09-22 00:00:00
  无相关新闻
查询时段: 2024-09-22 00:00:00 - 2024-09-23 00:00:00
  找到 3 条相关新闻
查询时段: 2024-09-23 00:00:00 - 2024-09-24 00:00:00
  找到 9 条相关新闻
查询时段: 2024-09-24 00:00:00 - 2024-09-25 00:00:00
  无相关新闻
查询时段: 2024-09-25 00:00:00 - 2024-09-26 00:00:00
  找到 5 条相关新闻
查询时段: 2024-09-26 00:00:00 - 2024-09-27 00:00:00
  无相关新闻
查询时段: 2024-09-27 00:00:00 - 2024-09-28 00:00:00
  无相关新闻
查询时段: 2024-09-28 00:00:00 - 2024-09-29 00:00:00
  无相关新闻
查询时段: 2024-09-29 00:00:00 - 2024-09-30 00:00:00
  无相关新闻
查询时段: 2024-09-30 00:00:00 - 2024-10-01 00:00:00
  找到 1 条相关新闻
查询时段: 2024-10-01 00:00:00 - 2024-10-02 00:00:00
  无相关新闻
查询时段: 2024-10-02 00:00:00 - 2024-10-03 00:00:00
  无相关新闻
查询时段: 2024-10-03 00:00:00 - 2024-10-04 00:00:00
  无相关新闻
查询时段: 2024-10-04 00:00:00 - 2024-10-05 00:00:00
  找到 1 条相关新闻
查询时段: 2024-10-05 00:00:00 - 2024-10-06 00:00:00
  无相关新闻
查询时段: 2024-10-06 00:00:00 - 2024